Fitness - fatigue modelling with Banister
====

This notebook deals with the Banister models \cite{banister1975systems, busso1994} on the evaluation and prediction of athlete performance in function of fatigue and fitness. The general equation is given below:

\begin{equation}
p_{t}=p_{0}+k_{a} \sum_{s=0}^{t-1} e^{-(t-s) / \tau_{a}} w_{s}-k_{f} \sum_{s=0}^{t-1} e^{-(t-s) / \tau_{f}} w_s
\label{eq:banister}
\end{equation}

* $p_{t}$ is the modelled performance at time $t$
* $p_0$ is the initial performance level
* $k_a$ and $k_f$ are fitness and fatigue magnitude factor
* $\tau_a$ and $\tau_f$ are fitness and fatigue time decay
* $w_s$ is the known training load per time unit, or ' the known training load per week (or day) from the first week of training to the week (or day) preceding the performance'

There are however some ***issues with the Banister model*** which are described in detail in \cite{Hellard2006}.
1. The model parameters suffer from poor identifiability, which seems logical as the mechanisms could correct for eachother. This feature leads to a lot of variability in the resulting parameter estimates. The model somehow has been reported to predict reasonably well. The model does have 4 parameters that can bring this in. 
2. On top of that there is usually only limited data, between 10 and 20 data points per athlete, to fit the model.
3. The Banister model parameters might not be so physically accurate, the model plausibility could be low.

There appears to be the upside that the model is stable and parameter estimates do not change much (on top of the other variability?) when a data point is left out.


Things to check on \cite{Hellard2006} still:
1. how was the model prediction assessed?
2. how was bootstrapping used?

# Preparations

## Imports
Python packages needed

In [ ]:
#basic
import pandas as pd
import numpy as np
import os, sys
# plotting
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline
import seaborn as sns
from matplotlib import cm
from palettable.scientific import sequential
from palettable import tableau
# calculations
from scipy.optimize import brute, differential_evolution
import multiprocessing
from pyswarm import pso
# sensitivity analysis
from SALib.analyze import sobol

## Visualisation settings

In [ ]:
# naming of parameters in plots
parname_dict = {
    'k_a' : '$k_a$',
    'k_f' : '$k_f$',
    'tau_a' : r'$\tau_a$',
    'tau_f' : r'$\tau_f$',
    'p_0' : '$p_0$',
}

In [ ]:
# overall plot settings
sns.set_style('whitegrid')
plt.rcParams['axes.labelsize'] = 16
plt.rcParams['axes.titlesize'] = 18
plt.rcParams['legend.fontsize'] = 16

# Experimental data

In [ ]:
# get all sporter dataset names
dataset_names = [f for f in os.listdir() if 'modellen.xlsx' in f]
sheetnames = ['Edwards', 'Banister', 'Lucia', 'sRPE', 'TSS']
dataset_dict = dict() #dictionary to store all data in
for d in dataset_names:
    # read sporter excel file and store in dictionary, referenced by sporter name
    dataset_dict[d[:-14]] = dict()
    for s in sheetnames:
        dataset_dict[d[:-14]][s] = pd.read_excel(d, sheet_name=s, index_col=0).loc[:,
                   ['Datum', 'TL', 'TT (W)', 'PTE', 'Fitness', 'NTE', 'Fatigue',
                   'TT performance', 'Performance', 'Modelfit']] #only store columns of interest

In [ ]:
#create figure
fig, ax  = plt.subplots(figsize=(12,3))
#select dataset
sporter = 'Wietse'
datatype = 'Lucia'
data2plot = dataset_dict[sporter][datatype]
#plot data
ax.plot(data2plot.loc[:,'TT (W)'],'*', label='Data '+sporter)
#format the figure
ax.set_xlim(0,100);
ax.set_xlabel('Time, in days');
ax.set_ylabel('Performance,\nin arbitrary units');
ax.legend();

# Model

## Banister model

\begin{equation}
p_{t}=p_{0}+k_{a} \sum_{s=0}^{t-1} e^{-(t-s) / \tau_{a}} w_{s}-k_{f} \sum_{s=0}^{t-1} e^{-(t-s) / \tau_{f}} w_s
\label{eq:banister2}
\end{equation}

Below is the implementation for Eq.\ref{eq:banister}. *The same notation is used, thus the meaning of the inputs to the model function* `ff_banister` *can be found under Eq.\ref{eq:banister}*.

### Implementation

In [ ]:
def sum_exp(t, k, tau, w_s):
    """calculation for summation exponential term (is this a convolution?)
    needed for the calculation of the fitness and fatigue terms"""
    retval = np.zeros((len(t),1))
    for i in range(1,len(t)-1):
        retval[i] = np.sum(np.exp(-(i-np.arange(i+1))/tau)*w_s[:i+1])
        
    return k*retval

def ff_banister(t, p_0, k_a, k_f, tau_a, tau_f, w_t, tt_W):
    """
    returns the performance at time t calculated with all the input parameters
    """
    p_t = p_0 + sum_exp(t, k_a, tau_a, w_t) - sum_exp(t, k_f, tau_f, w_t)
    
    for ti, tv in enumerate(tt_W):
        if tv>0.1:
            p_t[ti] = p_t[ti] + 2*(k_f-k_a)*w_t[ti-1]
            print(ti)
    
    return p_t

def kobes_way(t, p_0, k_a, k_f, tau_a, tau_f, w_t, tt_W):
    """
    try exactly the  code from the excel file
    """
    retval = np.zeros((len(t),1))
    retval[0] = p_0
    pte = np.zeros((len(t),1))
    nte = np.zeros((len(t),1))
    
    for i in range(1,len(t)):
        pte[i] = (pte[i-1]*np.exp(-1/tau_a) + w_t[i])
        nte[i] = (nte[i-1]*np.exp(-1/tau_f) + w_t[i])
        if tt_W[i]<0.1:
            retval[i] = p_0 + (pte[i])*k_a - (nte[i])*k_f
        else:
            retval[i] = p_0 + (pte[i-1]*np.exp(-1/tau_a))*k_a - (nte[i-1]*np.exp(-1/tau_f))*k_f
            
    simres = np.array((retval, pte, nte)).reshape(3, len(t)).transpose()
        
    return pd.DataFrame(simres, index=t, columns=['overall', 'fit', 'fat'])

### Retrieving data

Select an athlete out of Thibaux, Wietse or Tom

In [ ]:
# model inputs
sporter = 'Thibaux'
data4mod = dataset_dict[sporter]['TSS']
t = data4mod.index.values
data4mod[['TT (W)']] = data4mod[['TT (W)']].fillna(value=0)
data4mod[['TL']] = data4mod[['TL']].fillna(value=0)
w_t = data4mod[['TL']].values.ravel()
tt_w = data4mod[['TT (W)']].values.ravel()

### Verifying model implementation

Checking whether simulated outcomes are the same as those of Kobe V.

In [ ]:
# simulate the model using parameters from the excel and compare with Kobe's dank excel code
fig, ax = plt.subplots(figsize=(12,5))
mod_out = kobes_way(t, 240, 0.592627747, 0.830655943, 11.23205693, 6.327383027, w_t, tt_w)
plt.plot(t, data4mod.loc[:,'Modelfit'], label='Kobe')
plt.plot(mod_out['overall'] , '--', label='Michael')
# plt.plot(t, data4mod.loc[:,'TL'], '*', label='TL')
#plot data
ax.plot(t[tt_w>0.1], tt_w[tt_w>0.1],'*', label='Data '+sporter)
plt.legend();


## Spark and stop model

Or 'prikkel' and 'stop' model: calculation of the ideal times to respectively have the last big exercise and when to stop exercising prior to a contest (when the athlete should peak). Need to check this



This is calculated based on the fitter Banister model parameters

In [ ]:
# the model
def spark_stop(pars):
    """
    inputs:
        - pars {list}: contains the parameter values of the Banister model in the following order:
        [k_a, k_f, tau_a, tau_f]
    """
    # parameter values
    [k_a, k_f, tau_a, tau_f] = pars    
    spark = (tau_a*tau_f/(tau_a - tau_f)) * np.log((k_f*tau_a)/(k_a*tau_f))
    stop = (tau_a*tau_f/(tau_a - tau_f)) * np.log(k_f/k_a)
    return spark, stop

In [ ]:
spark_stop([0.634,0.716,9.541,7.812])

### Sensitivity analysis

In [ ]:
from SALib.analyze import sobol, delta
from SALib.sample import saltelli, latin
from SALib.plotting import bar, morris

In [ ]:
# Define the model inputs
problem = {
    'num_vars': 4,
    'names': ['k_a', 'k_f', 'tau_a', 'tau_f'],
    'bounds': [[0, 3],[0, 3],[0, 60],[0, 60]]
}

# Generate samples
param_values = latin.sample(problem, 10000)

# Run model (example)
Y = np.array([spark_stop(p) for p in param_values])

In [ ]:
# Perform analysis
Y_in = Y[:,1].copy()
good_values = ~((sobol_in<0) | (np.isinf(sobol_in)) | (np.isnan(sobol_in)))
Y_in = Y_in[good_values] 
X_in = param_values[good_values,:]
Si = delta.analyze(problem, X_in, Y_in, print_to_console=True)

## Optimisation
Visualise objective function and conduct a global optimisation

### Objective function

In [ ]:
# objective function
def mod_err_banister(pars, t_loc, w_t_loc, tt_w_loc, p_0, err_type='sse'):
    """
    pars {list}: contains the parameter values of the Banister model in the following order:
        [k_a, k_f, tau_a, tau_f]
    t_loc {np.array 1D}: time denotation (in days)
    w_t_loc {np.array 1D}: values from TL column in data
    tt_w_loc {np.array 1D}: values from TT (W) column in data
    err_type {string}: specifies model error calculation type, standard sum of squared errors
    """
    # parameter values
    [k_a, k_f, tau_a, tau_f] = pars
    # calculate model output
    mod_out = kobes_way(t_loc, p_0, k_a, k_f, tau_a, tau_f, w_t_loc, tt_w_loc)
    # reformat data for model error calculation
    data2compare = tt_w_loc.copy()
    data2compare[tt_w_loc<0.1] = np.nan
    # model error calculation
    if err_type == 'sse': #sum of squared errors
        mod_err = np.nansum(np.square(mod_out.overall.values - data2compare))
        
    return mod_err

### Actual optimisation

A brute force is carried out, which checks the model fit at a range of specified parameter value combinations. This is mainly to visualise the objective function.

In [ ]:
# parameter ranges 
par_ranges = (slice(0, 3, 0.1), slice(0, 3, 0.1),
             slice(0, 60, 1), slice(0,60,1))

par_ranges_dict = {
    'k_a' : np.arange(1e-4,3,step=0.1),
    'k_f' : np.arange(1e-4,3,step=0.1),
    'tau_a' : np.arange(0,60,step=1),
    'tau_f' : np.arange(0,60,step=1),
}

lowbounds = [l[0] for l in list(par_ranges_dict.values())]
uppbounds = [l[-1] for l in list(par_ranges_dict.values())]

In [ ]:
results = brute(mod_err_banister, par_ranges, args=(t, w_t, tt_w), full_output=True, disp=True)#, workers=32)
xeval = results[2]
feval = results[3]

#### Visualising results

In [ ]:
# how to plot this

fig_brutres = plt.figure(figsize=(10,7), constrained_layout=True)
kleurmap = cm.viridis_r #sequential.Bilbao_10.mpl_colormap
cont_nlev = 100

spec = gridspec.GridSpec(ncols=41, nrows=2, figure=fig_brutres)
ax11 = fig_brutres.add_subplot(spec[0, :20])
ax12 = fig_brutres.add_subplot(spec[0, 20:-1])
ax21 = fig_brutres.add_subplot(spec[1, :20])
ax22 = fig_brutres.add_subplot(spec[1, 20:-1])
ax_cb = fig_brutres.add_subplot(spec[:,-1])
# we plot each optimal objective function value for a combination of two selected parameters
# normalising to the same colour scale
normF = plt.Normalize(20, 100)

# plot optimal value found in function of combination of values of two parameters
# right now I have no idea but to rewrite this manually
# bc changing the parameter selection also changes the indexing

# k_a vs k_f
dims = [0,1]
parnames = [list(par_ranges_dict.keys())[i] for i in dims]
disc1 = np.shape(feval)[dims[0]]
disc2 = np.shape(feval)[dims[1]]
# storage of parameter value combinations
p1 = par_ranges_dict[parnames[0]]
p2 = par_ranges_dict[parnames[1]]
P1, P2 = np.meshgrid(p1, p2)
F = P1*2 +P2

#loop over first variable
for i in range(disc1):
    #loop over second variable
    for j in range(disc2):
        F[j,i] = np.sqrt(np.min(feval[i,j,:,:]))

# plot contour
mappab = ax11.contourf(P1, P2, F, levels=cont_nlev, norm=normF, cmap=kleurmap)
ax11.set_xlabel(parname_dict[parnames[0]])
ax11.set_ylabel(parname_dict[parnames[1]]);

# tau
dims = [2,3]
parnames = [list(par_ranges_dict.keys())[i] for i in dims]
disc1 = np.shape(feval)[dims[0]]
disc2 = np.shape(feval)[dims[1]]
# storage of parameter value combinations
p1 = par_ranges_dict[parnames[0]]
p2 = par_ranges_dict[parnames[1]]
P1, P2 = np.meshgrid(p1, p2)
F = P1*2 +P2

#loop over first variable
for i in range(disc1):
    #loop over second variable
    for j in range(disc2):
        F[j,i] = np.sqrt(np.min(feval[:,:,i,j]))

# plot contour
ax12.contourf(P1, P2, F, levels=cont_nlev, norm=normF, cmap=kleurmap)
ax12.set_xlabel(parname_dict[parnames[0]])
ax12.set_ylabel(parname_dict[parnames[1]]);


# fitness
dims = [0,2]
parnames = [list(par_ranges_dict.keys())[i] for i in dims]
disc1 = np.shape(feval)[dims[0]]
disc2 = np.shape(feval)[dims[1]]
# storage of parameter value combinations
p1 = par_ranges_dict[parnames[0]]
p2 = par_ranges_dict[parnames[1]]
P1, P2 = np.meshgrid(p1, p2)
F = P1*2 +P2

#loop over first variable
for i in range(disc1):
    #loop over second variable
    for j in range(disc2):
#         print(j)
        F[j,i] = np.sqrt(np.min(feval[i,:,j,:]))

# plot contour
ax21.contourf(P1, P2, F, levels=cont_nlev, norm=normF, cmap=kleurmap)
ax21.set_xlabel(parname_dict[parnames[0]])
ax21.set_ylabel(parname_dict[parnames[1]]);

# fatigue
dims = [1,3]
parnames = [list(par_ranges_dict.keys())[i] for i in dims]
disc1 = np.shape(feval)[dims[0]]
disc2 = np.shape(feval)[dims[1]]
# storage of parameter value combinations
p1 = par_ranges_dict[parnames[0]]
p2 = par_ranges_dict[parnames[1]]
P1, P2 = np.meshgrid(p1, p2)
F = P1*2 +P2

#loop over first variable
for i in range(disc1):
    #loop over second variable
    for j in range(disc2):
#         print(j)
        F[j,i] = np.sqrt(np.min(feval[:,i,:,j]))

# plot contour
ax22.contourf(P1, P2, F, levels=cont_nlev, norm=normF, cmap=kleurmap)
ax22.set_xlabel(parname_dict[parnames[0]])
ax22.set_ylabel(parname_dict[parnames[1]]);


# colorbar
cb = fig_brutres.colorbar(mappab, cax=ax_cb)
cb.set_label('absolute model error', fontsize=16)

fig_brutres.savefig(sporter+'_brutres.png', dpi=300)

### Global optimisation for every exercise data collection method

#### Particle swarm optimisation
The [particle swarm optimisation (PSO) technique](https://en.wikipedia.org/wiki/Particle_swarm_optimization) is a strategy for global optimisation. It is easy to conduct in parallel.

Constraints

In [ ]:
# function for inequality contraints on the optimisation
# in our case, k_f >= k_a
# make sure to have the exact same input arguments as in the optimisation function

def kcon(x, t, wt, ttw, p0):
    k_a = x[0]
    k_f = x[1]
    tau_a = x[2]
    tau_f = x[3]
    # calculate model so that fatigue does not go below 0
    mod_out = kobes_way(t, p0, k_a, k_f, tau_a, tau_f, wt, ttw)
    
    return [k_f-k_a, k_f*tau_f-1e-4, 0.5-np.sum(mod_out.overall.values<0)]

Conduct global optimisations with PSO per datatype available per athlete. It is even possible to perform a few optimisations per case, in order to increase the chance of finding optimal parameter values and to see which at which datatype the optimisation results in adequate and consistent optimal parameter values most often

In [ ]:
# conduct the PSO and store results in a python dictionary
dict_optires_pso = dict()
n_opts=8 # amount of optimisations to attempt per case
for si, sporter in enumerate(['Brecht']): #list(dataset_dict.keys())
    dict_optires_pso[sporter]=dict()
    for s in sheetnames:
        data4mod = dataset_dict[sporter][s]
        t = data4mod.index.values
        data4mod[['TT (W)']] = data4mod[['TT (W)']].fillna(value=0)
        data4mod[['TL']] = data4mod[['TL']].fillna(value=0)
        w_t = data4mod[['TL']].values.ravel()
        tt_w = data4mod[['TT (W)']].values.ravel()
        p0 = dataset_dict[sporter][s]['Performance'][0]
        dict_optires_pso[sporter][s] = pd.DataFrame(index = np.arange(n_opts), columns=list(par_ranges_dict.keys())+['f_opt'])
        
        # conduct a few optimisations
        for o in range(n_opts):
            pso_x, pso_f = pso(mod_err_banister, lowbounds, uppbounds, ieqcons=[], f_ieqcons=kcon, args=(t, w_t, tt_w, p0), kwargs={},
                   swarmsize=1280, omega=0.5, phip=0.5, phig=0.5, maxiter=50, minstep=1e-8, minfunc=1e-8, debug=False, processes=32)
            dict_optires_pso[sporter][s].iloc[o,:-1] = pso_x
            dict_optires_pso[sporter][s].iloc[o,-1] = pso_f

In [ ]:
# format results as a pandas dataframe
mind_optires_pso = pd.MultiIndex.from_product([list(dict_optires_pso.keys()), sheetnames, np.arange(n_opts)])
df_optires_pso = pd.DataFrame(index=mind_optires_pso, columns=list(par_ranges_dict.keys())+['f_opt'])

# get all optimisation results from the relevant dictionary
for sporter in list(dict_optires_pso.keys()):
    for s in sheetnames:
        df_optires_pso.loc[(sporter, s),:] = dict_optires_pso[sporter][s].values

# calculate absolute error, assuming model objective function was SSE
df_optires_pso.loc[:,'abs_err'] = (df_optires_pso.loc[:,'f_opt'].values)**(1/2)

##### Effect on spark and stop time

In [ ]:
# read data
# df_optires_pso_rd = pd.read_pickle('optires_pso.pkl')

df_optires_pso_rd = df_optires_pso.copy()
df_optires_pso_rd.head()


In [ ]:
# calculate prikkel and stop for every optimisation result
n_opt_tot = len(df_optires_pso_rd)
sparklist, stoplist = np.zeros((n_opt_tot,1)), np.zeros((n_opt_tot,1))
for r in range(n_opt_tot):
    try:
        spark, stop = spark_stop(df_optires_pso_rd.iloc[r,:4])
    except:
        spark, stop = np.nan, np.nan
    sparklist[r] = spark
    stoplist[r] = stop

df_optires_pso_rd.loc[:,'prikkel'] = sparklist  
df_optires_pso_rd.loc[:,'stop'] = stoplist   

##### Plot results

In [ ]:
df_optires_pso_rd

In [ ]:
datur = pd.read_excel('Brecht_modellen.xlsx', sheet_name='Overzicht parameters', skiprows=2, index_col=0)

In [ ]:
# model inputs
sporter = 'Brecht'
datatype = 'TSS'
opt_i = 2

#retrieve data
data4mod = dataset_dict[sporter][datatype]
t = data4mod.index.values
data4mod[['TT (W)']] = data4mod[['TT (W)']].fillna(value=0)
data4mod[['TL']] = data4mod[['TL']].fillna(value=0)
w_t = data4mod[['TL']].values.ravel()
tt_w = data4mod[['TT (W)']].values.ravel()
p0 = dataset_dict[sporter][datatype]['Performance'][0]
# create figure
fig, ax = plt.subplots(figsize=(12,5))
plt.plot(t, data4mod.loc[:,'Modelfit'], ':', label='result Kobe') # Kobe's result
kleuren = tableau.Tableau_10.mpl_colors
selection = [3,5,7] #range(8)
# plot optimisation results
for opt_i in selection: #range(n_opts):
    # parameter values
    optres_loc = df_optires_pso_rd.loc[sporter, datatype, opt_i]
    [k_a,k_f,tau_a,tau_f] = optres_loc.loc[['k_a','k_f', 'tau_a', 'tau_f']].values
    kleur = kleuren[opt_i]
    # line below: get parameters from data
    # [tau_a,tau_f,k_a,k_f] = datur.loc[datatype, ['t1','t2','k1','k2']].values

    mod_out = kobes_way(t, p0, k_a, k_f, tau_a, tau_f, w_t, tt_w)
    plt.plot(mod_out['overall'] , '--', color=kleur, label='result Michael '+str(opt_i), lw=3)
#     plt.plot(mod_out['fit'] , '-.', color=kleur, label='Fitness'+str(opt_i), lw=1)
#     plt.plot(mod_out['fat'] , ':', color=kleur, label='Fatigue'+str(opt_i), lw=1)

#plot data
ax.plot(t[tt_w>0.1], tt_w[tt_w>0.1],'*', label='Data '+sporter, markersize=10)
plt.legend(loc='upper right');
df_optires_pso_rd.loc[sporter, datatype, selection][['k_a', 'k_f','tau_a','tau_f','abs_err']]
# plt.ylim(-10,10)

## Wishlist

* model that relates parameters to optimal training time - model sensitivity
* validation: 
    * apply average model (average of fitted parameters?) to all datasets
    * leave one out (opletten welke punten)
* measurement method: check results global optimisation
    * consistency over sporters (tau should be similar for sporters)
    
    
**To do**
* refine grid: request however not possible, lowering one tenth... otherwise 324 trillion simulations
* Banister model global output for each method + effect on stop and spark time
    * global optimisation each method
    * sensitivity analysis on stop/spark time outcome
    

# References

(<a id="cit-banister1975systems" href="#call-banister1975systems">Banister, Calvert <em>et al.</em>, 1975</a>) Banister E W, Calvert T W, Savage M V <em>et al.</em>, ``_A systems model of training for athletic performance_'', Aust J Sport Med, vol. 7, number 3, pp. 57--61,  1975.

(<a id="cit-busso1994" href="#call-busso1994">Busso, Candau <em>et al.</em>, 1994</a>) Busso Thierry, Candau Robin and Lacour Jean Ren{\'{e}}, ``_Fatigue and fitness modelled from the effects of training on performance_'', Eur J Appl Physiol Occup Physiol, vol. 69, number 1, pp. 50--54, jan 1994.  [online](https://www.researchgate.net/publication/15242395)

(<a id="cit-Hellard2006" href="#call-Hellard2006">Hellard, Avalos <em>et al.</em>, 2006</a>) Hellard Philippe, Avalos Marta, Lacoste Lucien <em>et al.</em>, ``_Assessing the limitations of the Banister model in monitoring training_'', J Sports Sci, vol. 24, number 5, pp. 509--520,  2006.

